In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle


In [6]:
##Loaad the dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
##Preprocess the data
###Drop rownumber,id,surname

data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [8]:
label_encode_gender=LabelEncoder()
data['Gender']=label_encode_gender.fit_transform(data['Gender'])

In [9]:
## Onehot encode 'Geography'
from sklearn.preprocessing import OneHotEncoder
onehotencoder_geo=OneHotEncoder()
geo_encoder=onehotencoder_geo.fit_transform(data[['Geography']])
geo_encoder.toarray()


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [10]:
onehotencoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehotencoder_geo.get_feature_names_out(['Geography']))

In [12]:
##Combine one hot encoder columns
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
##Save the encoders and sscaler
with open ('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encode_gender,file)

with open ('label_encoder_geo.pkl','wb') as file:
    pickle.dump(onehotencoder_geo,file)

In [14]:
## Divide the dataset into independent and dependent features

X=data.drop('Exited',axis=1)
Y=data['Exited']

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)



In [15]:
with open ('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)



ANN Implementation


In [16]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [18]:
X_train.shape[1]

12

In [19]:
## Build our ANN Model

model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),   #Hidden Layer 1
    Dense(32,activation='relu'),   # Hidden Layer 2
    Dense(1,activation='sigmoid')            # Hidden Layer 3

])


d:\ML Course\DL Project\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,837 (34.52 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,892 (23.02 KB)

In [35]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
# loss=tf.keras.losses.BinaryCrossentropy()

In [43]:
## Model Compilation
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [44]:
## Setup the TensorBoard
timestamp = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = f"logs/fit/{timestamp}"
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [45]:
## Setup EarlyStopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [46]:
##Train the model
history= model.fit(
    X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8680 - loss: 0.3167 - val_accuracy: 0.8565 - val_loss: 0.3455
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8680 - loss: 0.3167 - val_accuracy: 0.8565 - val_loss: 0.3455
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8698 - loss: 0.3143 - val_accuracy: 0.8640 - val_loss: 0.3391
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8698 - loss: 0.3143 - val_accuracy: 0.8640 - val_loss: 0.3391
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8696 - loss: 0.3130 - val_accuracy: 0.8590 - val_loss: 0.3393
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8696 - loss: 0.3130 - val_accuracy: 0.8590 - val_loss: 0.3393
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8695 - loss: 0.3122 - val_accuracy: 0.8600 - val_loss: 0.3408
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8695 - loss: 0.3122 - val_accuracy: 0.8600

In [47]:
model.save('model.h5')

In [ ]:
# TensorBoard extension removed
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# TensorBoard display removed
%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6006 (pid 35972), started 0:15:51 ago. (Use '!kill 35972' to kill it.)